In [1]:
# ============================================================
# NYC Taxi Trip Duration Prediction (Colab Notebook)
# Steps 1 to 20 - End-to-End
# ============================================================

# =========================
# Step 1: ML Objective
# =========================
"""
Objective:
- Build a regression model to predict trip_duration (seconds)
- Use only features known before trip starts
- Minimize prediction error between actual & predicted trip_duration

Target:
- trip_duration (seconds)

Feature Constraints (Business-Driven):
- Exclude: dropoff_datetime (not known at trip start)
- Exclude: store_and_fwd_flag (excluded per requirement)
"""

# =========================
# Step 2: Expected Workflow
# =========================
"""
High-level workflow:
1) EDA
2) Data Cleaning & Validation
3) Feature Engineering
4) Model Training (Regression)
5) Evaluation (RMSE/MAE + business interpretation)
6) Insights + Deployment Readiness
"""

# =========================
# Step 3: Load Libraries
# =========================
import os
import json
import math
import zipfile
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from datetime import datetime

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor

import joblib

# Colab upload/download
from google.colab import files

print("✅ Libraries loaded.")


✅ Libraries loaded.


In [2]:
# =========================
# Step 4: Load Data (Upload CSV/Excel)
# =========================

uploaded = files.upload()

# Pick the first uploaded file
file_name = list(uploaded.keys())[0]
print("Uploaded file:", file_name)

# Read CSV or Excel
if file_name.lower().endswith(".csv"):
    df = pd.read_csv(file_name)
elif file_name.lower().endswith((".xlsx", ".xls")):
    df = pd.read_excel(file_name)
else:
    raise ValueError("❌ Please upload a .csv or .xlsx/.xls file")

print("✅ Data loaded successfully!")
print("Shape:", df.shape)
df.head()


Saving nyc_taxi_trip_duration.csv to nyc_taxi_trip_duration.csv
Uploaded file: nyc_taxi_trip_duration.csv
✅ Data loaded successfully!
Shape: (729322, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,29-02-2016 16:40,29-02-2016 16:47,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,11-03-2016 23:35,11-03-2016 23:53,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,21-02-2016 17:59,21-02-2016 18:26,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,05-01-2016 09:44,05-01-2016 10:03,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,17-02-2016 06:42,17-02-2016 06:56,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [3]:
# =========================
# Step 5: File Structure and Content
# =========================

print("Columns:\n", df.columns.tolist())
print("\nData Types:\n")
display(df.dtypes)

print("\nQuick Info:\n")
df.info()

print("\nSample rows:\n")
display(df.head(3))

# Check expected columns (common NYC Taxi Trip Duration dataset)
expected_cols = [
    "id", "vendor_id", "pickup_datetime", "dropoff_datetime",
    "passenger_count", "pickup_latitude", "pickup_longitude",
    "dropoff_latitude", "dropoff_longitude", "store_and_fwd_flag",
    "trip_duration"
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    print("\n⚠️ Missing expected columns (not always a problem if your dataset differs):", missing)
else:
    print("\n✅ All expected columns are present.")


Columns:
 ['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration']

Data Types:



,0
id,object
vendor_id,int64
pickup_datetime,object
dropoff_datetime,object
passenger_count,int64
pickup_longitude,float64
pickup_latitude,float64
dropoff_longitude,float64
dropoff_latitude,float64
store_and_fwd_flag,object



Quick Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  729322 non-null  object 
 1   vendor_id           729322 non-null  int64  
 2   pickup_datetime     729322 non-null  object 
 3   dropoff_datetime    729322 non-null  object 
 4   passenger_count     729322 non-null  int64  
 5   pickup_longitude    729322 non-null  float64
 6   pickup_latitude     729322 non-null  float64
 7   dropoff_longitude   729322 non-null  float64
 8   dropoff_latitude    729322 non-null  float64
 9   store_and_fwd_flag  729322 non-null  object 
 10  trip_duration       729322 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 61.2+ MB

Sample rows:



,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,29-02-2016 16:40,29-02-2016 16:47,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,11-03-2016 23:35,11-03-2016 23:53,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,21-02-2016 17:59,21-02-2016 18:26,2,-73.997314,40.721458,-73.948029,40.774918,N,1635



✅ All expected columns are present.


In [4]:
# =========================
# Step 6: Treating Missing Values
# =========================

missing_counts = df.isna().sum()
print("Missing values per column:\n")
display(missing_counts[missing_counts > 0])

print("\nTotal missing values in dataset:", int(df.isna().sum().sum()))

if int(df.isna().sum().sum()) == 0:
    print("✅ No missing values found (as per transcript).")
else:
    print("⚠️ Missing values found. We'll handle them in preprocessing pipeline.")


Missing values per column:



,0



Total missing values in dataset: 0
✅ No missing values found (as per transcript).


In [5]:
# =========================
# Step 7: Reformatting Features & Checking Consistency
# =========================

# Convert pickup/dropoff datetime
if "pickup_datetime" in df.columns:
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], errors="coerce")

if "dropoff_datetime" in df.columns:
    df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"], errors="coerce")

# Consistency check: dropoff - pickup should match trip_duration (if all exist)
if all(col in df.columns for col in ["pickup_datetime", "dropoff_datetime", "trip_duration"]):
    temp = df.dropna(subset=["pickup_datetime", "dropoff_datetime", "trip_duration"]).copy()
    if len(temp) > 0:
        calc = (temp["dropoff_datetime"] - temp["pickup_datetime"]).dt.total_seconds()
        diff = (calc - temp["trip_duration"]).abs()

        print("Consistency check (calculated duration vs trip_duration):")
        print("Rows checked:", len(temp))
        print("Max absolute difference (seconds):", float(diff.max()))
        print("Mean absolute difference (seconds):", float(diff.mean()))

        # show a few mismatches if any
        mismatches = temp.loc[diff > 1, ["pickup_datetime", "dropoff_datetime", "trip_duration"]].head(5)
        if len(mismatches) > 0:
            print("\n⚠️ Sample mismatches:")
            display(mismatches)
        else:
            print("✅ Values match (or extremely close) — target is reliable.")
    else:
        print("⚠️ Not enough valid rows for consistency check.")
else:
    print("⚠️ Cannot do consistency check (missing required columns).")


Consistency check (calculated duration vs trip_duration):
Rows checked: 729322
Max absolute difference (seconds): 59.0
Mean absolute difference (seconds): 19.982268463038274

⚠️ Sample mismatches:


,pickup_datetime,dropoff_datetime,trip_duration
0,2016-02-29 16:40:00,2016-02-29 16:47:00,400
1,2016-03-11 23:35:00,2016-03-11 23:53:00,1100
2,2016-02-21 17:59:00,2016-02-21 18:26:00,1635
4,2016-02-17 06:42:00,2016-02-17 06:56:00,848
5,2016-02-14 18:31:00,2016-02-14 18:55:00,1455


In [ ]:
# =========================
# Step 8: Target Exploration (trip_duration)
# =========================

if "trip_duration" not in df.columns:
    raise ValueError("❌ trip_duration column not found. This project needs trip_duration as target.")

# Basic stats
print("Trip Duration Stats (seconds):")
display(df["trip_duration"].describe())

# Convert to hours for exploration
df["trip_duration_hours"] = df["trip_duration"] / 3600.0

# Log transform (log1p to safely handle zeros if any)
df["trip_duration_log"] = np.log1p(df["trip_duration"])

# Plot raw distribution
plt.figure()
df["trip_duration"].clip(0, df["trip_duration"].quantile(0.99)).hist(bins=50)
plt.title("Trip Duration Distribution (clipped at 99th percentile)")
plt.xlabel("trip_duration (seconds)")
plt.ylabel("count")
plt.show()

# Plot log distribution
plt.figure()
df["trip_duration_log"].hist(bins=50)
plt.title("Trip Duration (log1p) Distribution")
plt.xlabel("log1p(trip_duration)")
plt.ylabel("count")
plt.show()

# Identify suspicious outliers
short_trips = (df["trip_duration"] < 10).sum()
long_trips = (df["trip_duration"] > 12*3600).sum()

print(f"Suspicious very short trips (<10s): {int(short_trips)}")
print(f"Extreme very long trips (>12 hours): {int(long_trips)}")


In [ ]:
# =========================
# Step 9: Univariate Visualization
# =========================

# Passenger count
if "passenger_count" in df.columns:
    plt.figure()
    df["passenger_count"].value_counts().sort_index().plot(kind="bar")
    plt.title("Passenger Count Distribution")
    plt.xlabel("passenger_count")
    plt.ylabel("count")
    plt.show()

# Vendor ID
if "vendor_id" in df.columns:
    plt.figure()
    df["vendor_id"].value_counts().sort_index().plot(kind="bar")
    plt.title("Vendor ID Distribution")
    plt.xlabel("vendor_id")
    plt.ylabel("count")
    plt.show()

# Store and forward flag (explore only; we'll exclude from model)
if "store_and_fwd_flag" in df.columns:
    plt.figure()
    df["store_and_fwd_flag"].value_counts(dropna=False).plot(kind="bar")
    plt.title("Store and Forward Flag Distribution")
    plt.xlabel("store_and_fwd_flag")
    plt.ylabel("count")
    plt.show()


In [ ]:
# =========================
# Step 10: Latitude & Longitude (clean geo outliers)
# =========================

geo_cols = ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]
missing_geo = [c for c in geo_cols if c not in df.columns]
if missing_geo:
    print("⚠️ Missing geo columns:", missing_geo)
else:
    # Basic NYC-ish bounds (approx)
    lat_min, lat_max = 40.4, 41.1
    lon_min, lon_max = -74.3, -73.6

    before = len(df)
    geo_mask = (
        df["pickup_latitude"].between(lat_min, lat_max) &
        df["dropoff_latitude"].between(lat_min, lat_max) &
        df["pickup_longitude"].between(lon_min, lon_max) &
        df["dropoff_longitude"].between(lon_min, lon_max)
    )
    df_geo = df[geo_mask].copy()
    after = len(df_geo)

    print(f"Geo filtering: {before} -> {after} rows kept (removed outliers).")

    # Plot pickup points (sample for speed)
    sample = df_geo.sample(n=min(20000, len(df_geo)), random_state=42)

    plt.figure()
    plt.scatter(sample["pickup_longitude"], sample["pickup_latitude"], s=1)
    plt.title("Pickup Locations (after geo cleaning, sampled)")
    plt.xlabel("pickup_longitude")
    plt.ylabel("pickup_latitude")
    plt.show()

    plt.figure()
    plt.scatter(sample["dropoff_longitude"], sample["dropoff_latitude"], s=1)
    plt.title("Dropoff Locations (after geo cleaning, sampled)")
    plt.xlabel("dropoff_longitude")
    plt.ylabel("dropoff_latitude")
    plt.show()


In [ ]:
# =========================
# Step 11: Bivariate Relations with Target
# =========================

# We'll use a cleaned version for clearer plots
df_work = df.copy()

# Remove extreme target outliers for plotting clarity (NOT yet for training)
low_q = df_work["trip_duration"].quantile(0.01)
high_q = df_work["trip_duration"].quantile(0.99)
df_plot = df_work[df_work["trip_duration"].between(low_q, high_q)].copy()

# Vendor vs duration (boxplot-like using grouped stats)
if "vendor_id" in df_plot.columns:
    display(df_plot.groupby("vendor_id")["trip_duration"].describe())

# Passenger count vs duration
if "passenger_count" in df_plot.columns:
    display(df_plot.groupby("passenger_count")["trip_duration"].describe())


In [ ]:
# =========================
# Step 12: Trip Duration vs Weekday
# =========================

if "pickup_datetime" in df.columns:
    df_work["pickup_weekday"] = df_work["pickup_datetime"].dt.day_name()
    df_work["pickup_weekday_num"] = df_work["pickup_datetime"].dt.weekday  # Mon=0

    weekday_stats = df_work.groupby("pickup_weekday_num")["trip_duration"].median().sort_index()
    print("Median trip duration by weekday (Mon=0 .. Sun=6):")
    display(weekday_stats)

    plt.figure()
    weekday_stats.plot(kind="bar")
    plt.title("Median Trip Duration by Weekday")
    plt.xlabel("weekday (Mon=0 ... Sun=6)")
    plt.ylabel("median trip_duration (seconds)")
    plt.show()
else:
    print("⚠️ pickup_datetime not available for weekday analysis.")


In [ ]:
# =========================
# Step 13: Vendor ID vs Trip Duration
# =========================

if "vendor_id" in df_work.columns:
    vendor_stats = df_work.groupby("vendor_id")["trip_duration"].describe()
    display(vendor_stats)

    plt.figure()
    df_plot.groupby("vendor_id")["trip_duration"].median().plot(kind="bar")
    plt.title("Median Trip Duration by Vendor")
    plt.xlabel("vendor_id")
    plt.ylabel("median trip_duration (seconds)")
    plt.show()
else:
    print("⚠️ vendor_id not available.")


In [ ]:
# =========================
# Step 14: Mean Trip Duration Vendor Wise
# =========================

if "vendor_id" in df_work.columns:
    mean_vendor = df_work.groupby("vendor_id")["trip_duration"].mean()
    print("Mean trip duration by vendor:")
    display(mean_vendor)
else:
    print("⚠️ vendor_id not available.")


In [ ]:
# =========================
# Step 15: Median Trip Duration Vendor Wise
# =========================

if "vendor_id" in df_work.columns:
    median_vendor = df_work.groupby("vendor_id")["trip_duration"].median()
    print("Median trip duration by vendor:")
    display(median_vendor)
else:
    print("⚠️ vendor_id not available.")


In [ ]:
# =========================
# Step 16: Trip Duration vs Passenger Count
# =========================

if "passenger_count" in df_work.columns:
    pc_stats = df_work.groupby("passenger_count")["trip_duration"].median().sort_index()
    print("Median duration by passenger count:")
    display(pc_stats)

    plt.figure()
    pc_stats.plot(kind="bar")
    plt.title("Median Trip Duration by Passenger Count")
    plt.xlabel("passenger_count")
    plt.ylabel("median trip_duration (seconds)")
    plt.show()
else:
    print("⚠️ passenger_count not available.")


In [ ]:
# =========================
# Step 17: Visualize Most Frequent Pickup Points (lat-lon map)
# =========================

if all(c in df.columns for c in ["pickup_latitude", "pickup_longitude"]):
    # Use rounded grid to approximate frequency (fast & simple)
    df_map = df.copy()
    df_map = df_map.dropna(subset=["pickup_latitude", "pickup_longitude"])
    df_map["p_lat_r"] = df_map["pickup_latitude"].round(3)
    df_map["p_lon_r"] = df_map["pickup_longitude"].round(3)

    freq = df_map.groupby(["p_lat_r", "p_lon_r"]).size().reset_index(name="count")
    freq = freq.sort_values("count", ascending=False)

    # Sample top N frequent points for visibility
    topN = min(3000, len(freq))
    freq_top = freq.head(topN).copy()

    # Color buckets: white(1-10), green(10-25), red(>25)
    def bucket(c):
        if c <= 10:
            return 1
        elif c <= 25:
            return 2
        else:
            return 3

    freq_top["bucket"] = freq_top["count"].apply(bucket)

    plt.figure()
    # plot each bucket separately to mimic colors without using seaborn
    b1 = freq_top[freq_top["bucket"] == 1]
    b2 = freq_top[freq_top["bucket"] == 2]
    b3 = freq_top[freq_top["bucket"] == 3]

    plt.scatter(b1["p_lon_r"], b1["p_lat_r"], s=2, label="1-10 trips")
    plt.scatter(b2["p_lon_r"], b2["p_lat_r"], s=4, label="10-25 trips")
    plt.scatter(b3["p_lon_r"], b3["p_lat_r"], s=6, label=">25 trips")

    plt.title("Frequent Pickup Zones (rounded grid, top points)")
    plt.xlabel("pickup_longitude (rounded)")
    plt.ylabel("pickup_latitude (rounded)")
    plt.legend()
    plt.show()

    print("Top 10 pickup zones by frequency:")
    display(freq.head(10))
else:
    print("⚠️ pickup_latitude/pickup_longitude not available.")


In [ ]:
# =========================
# Step 18: Correlation Heatmap (numerical)
# =========================

num_cols = df_work.select_dtypes(include=[np.number]).columns.tolist()
corr = df_work[num_cols].corr()

plt.figure(figsize=(10, 7))
plt.imshow(corr, aspect="auto")
plt.title("Correlation Matrix (Numerical Features)")
plt.xticks(range(len(num_cols)), num_cols, rotation=90)
plt.yticks(range(len(num_cols)), num_cols)
plt.colorbar()
plt.tight_layout()
plt.show()

# Show top correlations with trip_duration
if "trip_duration" in corr.columns:
    print("Top correlations with trip_duration:")
    display(corr["trip_duration"].sort_values(ascending=False).head(10))


In [ ]:
# =========================
# Step 19: (Modeling Prep): Feature Engineering
# =========================
# We will:
# - Keep only features available at trip start
# - Create time features from pickup_datetime
# - Create distance (Haversine) and bearing
# - Drop forbidden columns (dropoff_datetime, store_and_fwd_flag)

df_model = df.copy()

# Basic cleaning rules (common in NYC taxi duration)
# - remove weird passenger counts
if "passenger_count" in df_model.columns:
    df_model = df_model[df_model["passenger_count"].between(1, 6)]

# - remove suspicious duration outliers for training
#   (keep between 10 seconds and 3 hours by default; adjust if needed)
df_model = df_model[df_model["trip_duration"].between(10, 3*3600)]

# Drop forbidden columns from features (but keep target)
forbidden = []
if "dropoff_datetime" in df_model.columns:
    forbidden.append("dropoff_datetime")
if "store_and_fwd_flag" in df_model.columns:
    forbidden.append("store_and_fwd_flag")

# Time features
if "pickup_datetime" in df_model.columns:
    df_model["pickup_hour"] = df_model["pickup_datetime"].dt.hour
    df_model["pickup_day"] = df_model["pickup_datetime"].dt.day
    df_model["pickup_month"] = df_model["pickup_datetime"].dt.month
    df_model["pickup_weekday"] = df_model["pickup_datetime"].dt.weekday
else:
    print("⚠️ pickup_datetime missing: time features will be skipped.")

# Haversine distance + bearing
def haversine_km(lat1, lon1, lat2, lon2):
    # Earth radius (km)
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def bearing_deg(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    y = np.sin(dlon) * np.cos(lat2)
    x = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(dlon)
    brng = np.degrees(np.arctan2(y, x))
    return (brng + 360) % 360

geo_ready = all(c in df_model.columns for c in ["pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude"])
if geo_ready:
    df_model["distance_km"] = haversine_km(
        df_model["pickup_latitude"], df_model["pickup_longitude"],
        df_model["dropoff_latitude"], df_model["dropoff_longitude"]
    )
    df_model["bearing"] = bearing_deg(
        df_model["pickup_latitude"], df_model["pickup_longitude"],
        df_model["dropoff_latitude"], df_model["dropoff_longitude"]
    )
else:
    print("⚠️ Geo columns missing: distance/bearing will be skipped.")

# Log target (often improves regression)
df_model["trip_duration_log"] = np.log1p(df_model["trip_duration"])

print("✅ Modeling dataset shape after cleaning:", df_model.shape)
df_model.head()


In [ ]:
# =========================
# Step 19.1: (Modeling): Train/Test Split
# =========================

target = "trip_duration"
target_log = "trip_duration_log"

# Define feature columns:
# Keep everything except:
# - target columns
# - forbidden columns
# - id (not predictive)
drop_cols = ["id", target, target_log] + forbidden

feature_cols = [c for c in df_model.columns if c not in drop_cols]

X = df_model[feature_cols].copy()
y = df_model[target].copy()
ylog = df_model[target_log].copy()

X_train, X_test, y_train, y_test, ylog_train, ylog_test = train_test_split(
    X, y, ylog, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("Features used:", len(feature_cols))


In [ ]:
# =========================
# Step 19.2: (Modeling): Preprocessing Pipeline
# =========================
# - Numeric: impute median
# - Categorical: impute most_frequent + onehot

numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = [c for c in X_train.columns if c not in numeric_features]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)


In [ ]:
# =========================
# Step 19.3: (Model Training): Baseline + Models
# =========================

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

models = {
    "Baseline_Median": DummyRegressor(strategy="median"),
    "Ridge": Ridge(alpha=2.0, random_state=42),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(
        n_estimators=250, random_state=42, n_jobs=-1, max_depth=None
    )
}

results = []

trained_pipelines = {}

for name, model in models.items():
    pipe = Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", model)
    ])
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)

    mae = mean_absolute_error(y_test, pred)
    r = rmse(y_test, pred)

    # also compute RMSE on log scale (for business realism of % errors)
    pred_log = np.log1p(np.maximum(pred, 0))
    r_log = rmse(ylog_test, pred_log)

    results.append({
        "model": name,
        "MAE_seconds": mae,
        "RMSE_seconds": r,
        "RMSE_log1p": r_log
    })

    trained_pipelines[name] = pipe

results_df = pd.DataFrame(results).sort_values("RMSE_seconds")
print("✅ Model evaluation results (sorted by RMSE):")
display(results_df)


In [ ]:
# =========================
# Step 19.4: (Choose Best Model)
# =========================

best_model_name = results_df.iloc[0]["model"]
best_pipe = trained_pipelines[best_model_name]

print("✅ Best model selected:", best_model_name)

# Predictions dataframe (test set)
test_pred = best_pipe.predict(X_test)
pred_df = X_test.copy()
pred_df["actual_trip_duration"] = y_test.values
pred_df["predicted_trip_duration"] = test_pred
pred_df["abs_error_seconds"] = (pred_df["actual_trip_duration"] - pred_df["predicted_trip_duration"]).abs()
pred_df["error_percent"] = (pred_df["abs_error_seconds"] / pred_df["actual_trip_duration"]) * 100

display(pred_df.head(10))

# Quick error distribution plot
plt.figure()
pred_df["abs_error_seconds"].clip(0, pred_df["abs_error_seconds"].quantile(0.99)).hist(bins=50)
plt.title(f"Absolute Error Distribution (seconds) - {best_model_name} (clipped 99th)")
plt.xlabel("abs_error_seconds")
plt.ylabel("count")
plt.show()


In [ ]:
# =========================
# Step 20: Conclusion (Auto Insights)
# =========================

conclusion = {
    "best_model": best_model_name,
    "rows_used_for_training": int(len(df_model)),
    "features_used_count": int(len(feature_cols)),
    "notes": [
        "Model trained after removing extreme duration outliers (10s to 3 hours) and passenger_count outside 1-6.",
        "Time features + distance (Haversine) + bearing often improve prediction.",
        "Log-transform of target is useful for stabilizing long-tail trip durations."
    ]
}

print("✅ Conclusion Summary:")
print(json.dumps(conclusion, indent=2))


In [ ]:
# =========================
# Step 21: Success Metrics (Technical & Business)
# =========================
# You can tune these thresholds based on your business expectations.

best_row = results_df[results_df["model"] == best_model_name].iloc[0]

success_criteria = pd.DataFrame([
    {
        "Metric": "RMSE_seconds",
        "Target (example)": "<= 500 sec",
        "Current": float(best_row["RMSE_seconds"]),
        "Business Meaning": "Average error should be within ~8 minutes for dispatch planning."
    },
    {
        "Metric": "MAE_seconds",
        "Target (example)": "<= 300 sec",
        "Current": float(best_row["MAE_seconds"]),
        "Business Meaning": "Typical error should be within ~5 minutes for real-time assignment."
    },
    {
        "Metric": "Error % (median)",
        "Target (example)": "<= 25%",
        "Current": float(pred_df["error_percent"].median()),
        "Business Meaning": "Most trips should be predicted within a quarter of actual duration."
    },
    {
        "Metric": "Error % (p90)",
        "Target (example)": "<= 60%",
        "Current": float(pred_df["error_percent"].quantile(0.90)),
        "Business Meaning": "Even hard trips should not be wildly off most of the time."
    }
])

print("✅ Success Criteria:")
display(success_criteria)


In [ ]:
# =========================
# FINAL STEP: Export outputs for download (ZIP)
# =========================

# 1) Predictions file
pred_out = "1_NYC_Taxi_Trip_Duration_Predictions.csv"
pred_df.to_csv(pred_out, index=False)

# 2) Metrics file
metrics_out = "2_Model_Evaluation_Metrics.csv"
results_df.to_csv(metrics_out, index=False)

# 3) Success criteria file
success_out = "3_Success_Criteria.csv"
success_criteria.to_csv(success_out, index=False)

# Save trained model
model_out = "trained_model.joblib"
joblib.dump(best_pipe, model_out)

# Save feature list
features_out = "feature_columns.json"
with open(features_out, "w") as f:
    json.dump(feature_cols, f, indent=2)

# Zip everything
zip_name = "NYC_Taxi_Trip_Duration_Export.zip"
with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as z:
    for fn in [pred_out, metrics_out, success_out, model_out, features_out]:
        z.write(fn)

print("✅ Export files created:")
print("-", pred_out)
print("-", metrics_out)
print("-", success_out)
print("-", model_out)
print("-", features_out)
print("\n✅ Zipped into:", zip_name)

# Trigger download
files.download(zip_name)
